In [ ]:
import numpy as np
import pandas as pd
import requests
import time

from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt

In [ ]:
def parse_timestamp(timestamp):
    if 'ago' in timestamp:
        return (pd.Timestamp.today() - pd.Timedelta(timestamp.replace(' ago', ''))).date()
    else:
        return pd.to_datetime(timestamp).date()

In [ ]:
states = pd.read_csv('state.csv')['Code']
can_provinces = pd.read_csv('can_province.csv')['Code']

In [ ]:
review_scores = []
review_titles = []
review_locations = []
review_users = []
review_timestamps = []
    
page = 1
start = time.time()

while True:

    re = requests.get(f'https://onebite.app/reviews?page={page}')
    soup = BeautifulSoup(re.text, 'html.parser')

    if soup.select("div[class*='errorContainer']"):
        print(f'error found on page {page}')
        page += 1
        continue

    review_scores_elements = soup.select("p[class*='rating__score']")
    review_titles_elements = soup.select("h2[class*='reviewCard__title']")
    review_locations_elements = soup.select("p[class*='reviewCard__location']")
    review_users_elements = soup.select("p[class*='userMeta__username']")
    review_timestamps_elements = soup.select("p[class*='userMeta__timestamp']")

    for score in review_scores_elements:
        review_scores.append(float(score.text))

    for title in review_titles_elements:
        review_titles.append(title.text)

    for location in review_locations_elements:
        review_locations.append(location.text)

    for user in review_users_elements:
        review_users.append(user.text.replace('Verified', ''))

    for timestamp in review_timestamps_elements:
        review_timestamps.append(parse_timestamp(timestamp.text))

    if not soup.select("a[class*='btn--next']"):
        break
        
    if page % 1000 == 0:
        print(page, re.text[:5])

    page += 1

end = time.time()
end - start

In [ ]:
all_reviews_info = list(zip(review_titles, review_locations, review_users, review_timestamps, review_scores))

df = pd.DataFrame(all_reviews_info, columns=['title', 'location', 'user', 'timestamp', 'rating'])

In [ ]:
df['city'] = df['location'].apply(lambda x: x.split(', ')[0])
df['state-prov'] = df['location'].apply(lambda x: x.split(', ')[1])
df.drop(columns=['location'], inplace=True)

df = df[df['state-prov'].isin(states) | df['state-prov'].isin(can_provinces)]

df = df[['title', 'city', 'state-prov', 'user', 'timestamp', 'rating']]

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.to_csv('onebite_ratings.csv')

In [ ]:
def find_last_page(lower_search_bound=12750):
    
    page = lower_search_bound

    while True:

        url = f'https://onebite.app/reviews?page={page}'

        re = requests.get(url)
        soup = BeautifulSoup(re.text, 'html.parser')

        if not soup.select("a[class*='btn--next']"):
            return page
        
        page += 1
        
def scrape(url):
    print(url)
    review_scores = []
    review_titles = []
    review_locations = []
    review_users = []
    review_timestamps = []

    re = requests.get(url)
    soup = BeautifulSoup(re.text, 'html.parser')

    review_scores_elements = soup.select("p[class*='rating__score']")
    review_titles_elements = soup.select("h2[class*='reviewCard__title']")
    review_locations_elements = soup.select("p[class*='reviewCard__location']")
    review_users_elements = soup.select("p[class*='userMeta__username']")
    review_timestamps_elements = soup.select("p[class*='userMeta__timestamp']")

    for score in review_scores_elements:
        review_scores.append(float(score.text))

    for title in review_titles_elements:
        review_titles.append(title.text)

    for location in review_locations_elements:
        review_locations.append(location.text)

    for user in review_users_elements:
        review_users.append(user.text.replace('Verified', ''))

    for timestamp in review_timestamps_elements:
        review_timestamps.append(parse_timestamp(timestamp.text))
    
    
    return review_titles, review_locations, review_users, review_timestamps, review_scores

In [ ]:
## Multithreading

urls = [f'https://onebite.app/reviews?page={page}' for page in range(1, find_last_page() + 1)]

columns = {'title', 'location', 'user', 'date', 'rating'}

df = pd.DataFrame(columns=columns)

with ThreadPoolExecutor(max_workers=100) as p:

    result = p.map(scrape, urls)

    for page_content in result:

        df = pd.concat([df, pd.DataFrame(list(zip(*page_content)), columns=columns)])

df.reset_index(inplace=True)

In [ ]:
df.rename(columns={'date': 'timestamp'}, inplace=True)
df['city'] = df.location.apply(lambda x: x.split(', ')[0])
df['state-prov'] = df.location.apply(lambda x: x.split(', ')[1])

In [ ]:
df.drop(columns={'location'}, inplace=True)

In [ ]:
df = df[['title', 'city', 'state-prov', 'user', 'timestamp', 'rating']]

In [ ]:
df.to_csv('onebite_ratings.csv')